In [ ]:
import numpy as np
import scipy.linalg as lin
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter

## Function to generate sorted eigenvalues (evals) from a matrix (M) of any dimension

def get_evals(M):

    ## Check if matrix is square

    rows, cols = M.shape
    if rows != cols:
        raise ValueError("Matrix must be square (N x N)")

    ## Compute eigenvalues using NumPy's linalg

    evals, evecs = np.linalg.eig(M)
    return np.round(sorted(evals.real), 3)

## Initialise dictionary of molecular connections to determine huckel matrices

other_conn = {}

## Using Python's NetworkX to generate platonic solid graphs
## Convert to connections

solids = ["tetrahedron", "cube", "octahedron", "dodecahedron", "icosahedron"]

for s in solids:
    if s == "tetrahedron":
        G = nx.tetrahedral_graph()
    elif s == "cube":
        G = nx.cubical_graph()
    elif s == "octahedron":
        G = nx.octahedral_graph()
    elif s == "icosahedron":
        G = nx.icosahedral_graph()
    elif s == "dodecahedron":
        G = nx.dodecahedral_graph()

    conn_list = [list(G.neighbors(n)) for n in range(len(G.nodes))]
    atoms = ["C"] * len(G.nodes)

    other_conn[s] = (conn_list, atoms)

# Now append dictionary with naphthalene (manual)

other_conn["naphthalene"] = (
    [[1,9],[0,2],[1,3,7],[2,4],[3,5],
     [4,6],[5,7],[2,6,8],[7,9],[8,0]],
    ["C"] * 10
)

print(other_conn.keys())

## Function to generate Huckel matrix from a length n linear polyene

def huckel_linear(N, alpha, beta):

    ## Initialise huckel matrix with correct no. dimensions

    H = np.zeros((N, N))

    ## Looping over all rows and adding alphas and betas in suitable places

    for i in range(N):
        H[i, i] = alpha
        if i > 0:
            H[i, i-1] = beta
    H += np.transpose(H)

    return H

## Function to generate Huckel matrix from a length n cyclic polyene

def huckel_cyclic(N, alpha, beta):

    H = huckel_linear(N, alpha, beta)

    ## Adding betas where ring loops round

    H[0, N-1] = beta
    H[N-1, 0] = beta

    return H

## Function to generate Huckel matrix from a specified platonic solid

def huckel_other(molecule, alpha=0, beta=-1):

    ## Make input case-insensitive for matching to platonic_conn dict

    molecule = molecule.lower()

    ## Check string is present in platonic_conn dict

    if molecule not in other_conn:
        raise ValueError("Solid must be one of: napthalene, tetrahedron, cube, octahedron, dodecahedron, icosahedron")

    ## Initialise huckel matrix with correct no. dimensions

    conn_list, atoms_list = other_conn[molecule]
    N = len(conn_list)
    H = np.zeros((N, N))

    ## Loop over nested list in key-value pair

    for i in range(N):
        H[i, i] = alpha
        for atom_index in conn_list[i]:
            H[i, atom_index] = beta

    return H

## CLI

while True:
    try:
        alpha = float(input("Enter α (Coulomb integral, default 0): ") or 0)
        break
    except ValueError:
        print("Invalid input. Enter a number.")

while True:
    try:
        beta = float(input("Enter β (Resonance integral, default -1): ") or -1)
        break
    except ValueError:
        print("Invalid input. Enter a number.")

# Get system type from user

while True:
    system_type = input("Enter system type ('linear', 'cyclic', or 'other'): ").strip().lower()
    if system_type in ["linear", "cyclic", "other"]:
        break
    print("Invalid system type. Please enter 'linear', 'cyclic', or 'other'.")

# Get number of atoms or molecule name from user

if system_type in ["linear", "cyclic"]:
    while True:
        try:
            N = int(input("Enter the number of atoms (N): "))
            if N < 1:
                print("N must be at least 1.")
                continue
            break
        except ValueError:
            print("Invalid input. Enter an integer.")
    solid_name = None
    if system_type == "linear":
        energies = get_evals(huckel_linear(N, alpha, beta))
    else:
        energies = get_evals(huckel_cyclic(N, alpha, beta))
    num_electrons = N
else:
    while True:
        solid_name = input("Enter Platonic solid (tetrahedron, cube, octahedron, dodecahedron, icosahedron, naphthalene): ").strip().lower()
        if solid_name in other_conn:
            break
        print(f"Invalid solid name. Choose from: {', '.join(other_conn.keys())}")

    ## Determine N from connectivity list

    N = len(other_conn[solid_name][0])
    energies = get_evals(huckel_other(solid_name, alpha, beta))
    num_electrons = len(other_conn[solid_name][0])


if system_type != "other":
    print(f"\nHuckel energies for a {system_type} system with N={N}:")

else:
    print(f"\nHuckel energies for {solid_name}:")

## Initialise variables to determine markers next to energy level

rounded_energies = [round(e, 6) for e in energies]

energy_counts = Counter(rounded_energies)

sorted_energies = sorted(energy_counts.keys())

print("Energy (α + xβ)     Degeneracy     Occupancy")
print("--------------------------------------------")

## Build occupancy marker by looping over electron count

for e_val in sorted_energies:
    deg = energy_counts[e_val]

    max_electrons = 2 * deg

    electrons_in_level = min(num_electrons, max_electrons)
    num_electrons -= electrons_in_level

    marker = "↑↓" * (electrons_in_level // 2)
    if electrons_in_level % 2:
        marker += "↑"

    print(f"{e_val:8.4f}           {deg:3d}              {marker}")